In [177]:
import pandas as pd
from collections import Counter
from shapely.geometry import Point, MultiPoint
from dateutil import parser
import geopandas as gp
import hdbscan
import matplotlib.pyplot as plt
import time
%matplotlib inline

In [332]:
#import seaborn as sns
#sns.set_context('poster')
#sns.set_color_codes()
lymeData

text  NLPCat  \
id                                                                              
461673769212207100  A1 #sblchat zero's are for playing tick tack t...       1   
461673974708334500                                Tick, tick, tick...       1   
461684743994884000  Riley on a trail run. Two large thorns, a tick...       3   
461702279247458300  I Hate People Who Smack On There Food While Ea...       1   
461706893459599300  @ATweeter_7cb1dad413ed6bb91745ee426427b42a05aa...       1   
461708118783569900  Now convinced I have Lyme disease and/or a tic...       4   
461708908621348800  I'm not perfect. I'll annoy you, tick you off ...       3   
461709466896191400                Any lil thing will just tick me off       1   
461710750541942700  I miss not even one week and I miss Worldstar ...       3   
461710918611509200  Sentry 3 Count FiproGuard Plus Dog Flea and Ti...       1   
461710923317534700  Sentry 3 Count FiproGuard Plus Dog Flea and Ti...       1   
461710927704776700  Sentry 6 Count FiproGuard Plus Dog Flea and Ti...       1   
461710932276563900  Sentry 6 Count FiproGuard Plus Dog Flea and Ti...       1   
461712547016482800             tick tick tick tick #GoAvsGo #WhyNotUs       1   
461712705884545000           Those seconds tick by too slow. #LAKings       1   
461713096013144000  Enter Contest and Win Body Scrub. Tick Tock! h...       3   
461714589529300900  10 mins left to take advantage of this amazing...       1   
461714775622164400  Lyme Disease, Ehrlichiosis &amp; Babesia: A Re...       4   
461714828998889400                              hooo datz a tick unn.       1   
461715884147359700  TICK TICCCCCCCK TICK TICCCCCK TICK TICK Everyo...       1   
461715990301376500  " I was dug in like a tick but I jumped out li...       1   
461716052385071100  @ATweeter_4143924ad8c8278e441a0051a6b00fb726e8...       1   
461716675667443700  AVMA: Safe use of flea and tick preventive pro...       1   
461716677629976500  AVMA: Safe use of flea and tick preventive pro...       1   
461716828738162600                       tick tick tick tick #GoAvsGo       1   
461716835281281000    Dir en grey theGazettE cali=gari Plastic Tre...       1   
461717064479031200                          Tick is a fucking dumbass       1   
461717160369209300  IF YOU'RE GOING TO STAND AROUND AND WATCH THE ...       1   
461717849157804000  I'm not perfect. I'll annoy you, tick you off ...       3   
461718272715411400  My boy was all "tick me in daddy, I go to bed....       3   
...                                                               ...     ...   
751235697743278000  Shoutout to my tick thighs for always acting a...       3   
751253775449333700  I wanted to find a pokemon in my backyard and ...       3   
751270548454273000  Will my children one day also lay awake at nig...       3   
751426672491171800  How to remove a tick the RIGHT WAY (Its not wh...       3   
751437074901893100  Remember that time I found a tick in my bed? T...       3   
751519652312866800  Pretty sure I got a tick today and almost died...       3   
751614431579299800  Yooo someone take me off Xfinity Center. I'd r...       3   
751624563880972200                        When you find a tick on you       3   
751806517012115400                          i pull up wit a drum$tick       3   
751821318123843500  Happy to obnoxiously fly this and tick off the...       3   
751823482200133600  Another day, another roadtripping, another tic...       3   
751894288087277500  a museum visitor just spent thirty minutes tel...       3   
751965469733556200  Went to go to bed and found a tick on my thigh...       3   
751968775440826300  Talbot will do just about anything to avoid an...       3   
751976894539534300  Just pulled tick #4 off Stella from this sprin...       3   
751991435931639800  Ima be single till I find someone that's rlly ...       3   
751998911590436800  I found a tick crawling up my shoulder this af...       3   
752125963073314

In [316]:
lymeTweetFile = "studies/lymeStudy/search/lymeStudy_CollectedTweets.csv"
lymeData = pd.read_csv(lymeTweetFile)
secsPerDay = 60*60*24
lymeData['UTCTime'] = [float(parser.parse(i).strftime("%s"))/secsPerDay for i in lymeData['created_at']]
lymeData = lymeData.set_index(['id'])
biteData = lymeData[lymeData['NLPCat']==3]
biteLocs = lymeData[['lat','lon','UTCTime',]].dropna(how='any')
(lon1,lat1,lon2,lat2) = (-83.68,36.54,-75.23,39.47)
biteLocs = biteLocs[(biteLocs['lat'] > lat1) & 
                   (biteLocs['lat'] < lat2) &
                   (biteLocs['lon'] > lon1) &
                   (biteLocs['lon'] < lon2)]
biteLocs

lat        lon       UTCTime
id                                                    
461720487542796200  37.047890 -80.779790  16191.385613
461785854311424000  38.419240 -82.445100  16191.565984
461799790833958900  39.045750 -76.641270  16191.604444
461836320743387100  39.045750 -76.641270  16191.705255
461836784649777100  38.984600 -77.094700  16191.706528
461839688366837700  39.045750 -76.641270  16191.714537
461842447749099500  39.045750 -76.641270  16191.722153
461862041725054900  37.122360 -76.345770  16191.776227
461870045887725500  38.958630 -77.357000  16191.798310
461870239429713900  38.990660 -77.026000  16191.798843
461870525007278000  38.958630 -77.357000  16191.799630
461872825117065200  38.990660 -77.026000  16191.805984
461880643132342200  39.099270 -76.848300  16191.827558
461895504402857900  39.045750 -76.641270  16191.868565
461914098989801400  37.540720 -77.436040  16191.919873
461939851453947900  37.122360 -76.345770  16191.990938
461953646989541300  39.290380 -76.612180  16192.029005
461953958878404600  39.031950 -76.908820  16192.029873
461998601968058300  38.990660 -77.026000  16192.153056
462014303756095400  37.087080 -76.473010  16192.196389
462017328776224700  39.045750 -76.641270  16192.204734
462051938167967700  37.540720 -77.436040  16192.300243
462075600707264500  38.029300 -78.476670  16192.365532
462186027739000800  36.850760 -76.285870  16192.670255
462200161960468400  38.442160 -76.743970  16192.709259
462201367294377900  39.401490 -76.601910  16192.712581
462216073707012000  38.736723 -77.569319  16192.753171
462232131612061600  39.290380 -76.612180  16192.797477
462233336673734600  39.290380 -76.612180  16192.800810
462249545254985700  36.948450 -81.084800  16192.845532
...                       ...        ...           ...
742535200605937600  37.122360 -76.345770  16966.286597
743111433513095100  38.804830 -77.046920  16967.876690
743130852222238700  38.844520 -82.137080  16967.930278
743457000567123900  38.923622 -77.052507  16968.830278
743501585414819800  37.431570 -78.656890  16968.953310
743911213378084800  37.516470 -82.806710  16970.083669
744005608421089200  37.458900 -77.468500  16970.344144
744409621935300600  38.478410 -82.637930  16971.459005
745407353684533200  39.290380 -76.612180  16974.212222
745407372454158300  37.431570 -78.656890  16974.212280
746514773379006400  38.991590 -76.941510  16977.268125
747132290342526900  39.158100 -75.524360  16978.972141
747421700724264900  38.902900 -77.047940  16979.770764
747455227415113700  38.907190 -77.036870  16979.863275
747623323555749800  38.749200 -77.448880  16980.327141
747829566253707200  38.907190 -77.036870  16980.896262
748244418084352000  37.413750 -79.142240  16982.041030
748976597143740400  38.881070 -77.001227  16984.061458
748987730592296900  38.907190 -77.036870  16984.092187
749026635165425600  39.006770 -76.779130  16984.199537
749098462457712600  39.045750 -76.641270  16984.397743
750774555132698600  39.045750 -76.641270  16989.022882
750809608126656500  39.045750 -76.641270  16989.119606
750813124953985000  37.422340 -81.016190  16989.129317
751056296267354100  38.902900 -77.047940  16989.800336
751253775449333700  37.431570 -78.656890  16990.345278
751426672491171800  39.032050 -76.502730  16990.822384
751614431579299800  39.003160 -76.971000  16991.340498
751976894539534300  37.270970 -79.941420  16992.340706
752496015316553700  38.597620 -80.454900  16993.773206

[3121 rows x 3 columns]

In [351]:
def getClusters(data, kwds,cfg='null'):
    labels = hdbscan.HDBSCAN(**kwds).fit_predict(data)
    return labels
    

def prepClusters(data,labels):
    workData = data
    workData['label'] = labels
    counted = dict(Counter(list(labels)))
    points = []
    hulls = []
    centX = []
    centY = []
    indexOut = []
    counts = []
    
    for (i,j) in sorted(counted.items(), key=lambda x: x[1], reverse=True):
        if i != -1:
            cluster = workData[workData['label'] == i]
            clusterCoords = zip(cluster['lat'],cluster['lon'])
            clusterPoints = MultiPoint([Point(k[0],k[1]) for k in clusterCoords])
            points.append(clusterPoints)
            hulls.append(clusterPoints.convex_hull)
            centroid = hulls[-1].centroid
            centX.append(centroid.x)
            centY.append(centroid.y)
            indexOut.append(i)
            counts.append(j)

    gdf = gp.GeoDataFrame()
    gdf['points'] = points
    gdf['hulls'] = hulls
    gdf['centX'] = centX
    gdf['centY'] = centY
    gdf['keys']= indexOut
    gdf['count'] = counts
    gdf.geometry=gdf['hulls']
    return gdf


def plotClusters(shapes,pts,show=8):
    fig,ax = plt.subplots()
    top9 = shapes.head(n=show).iloc[::-1]
    

    top9.plot(column='count',
              ax=ax,
              cmap='YlOrRd',
              legend=True,
              scheme='QUANTILES',
              k=show,
              alpha=0.5)
    
    if show < 10:
        leg = ax.get_legend()
        for i in reversed(range(show)):
            leg.get_texts()[show-i-1].set_text("%s: %s" % (i,top9['count'][i]))
            plt.text(top9['centX'][i],top9['centY'][i],i,
                     bbox={'facecolor':'white', 'alpha':0.5, 'pad':4, 'edgecolor':'none'})
        leg.set_bbox_to_anchor((1, 1))
    plt.scatter(pts['lat'],pts['lon'],s=5,alpha=0.3,color='r')
    return list(top9.index)


def getTextSubsets(tweets,data,dispKey):
    #textSubSets = {i:pd.merge(data[data['label'] == j],tweets[['text']]) for i,j in dispKey.iteritems()}
    textSubSets = {i:pd.concat([data[data['label'] == j],tweets[['text']]],axis=1,join='inner') for i,j in dispKey.iteritems()}
    return textSubSets
    for i,j in textSubSets.iteritems:
        None
    textSubSets = dict()
    for i,j in dispKey.iteritems():
        dfSub = data[data['label'] == j]
        textSubSets[i] = dfSub
    return textSubSets
    

def runClusters(tweets,data,cfg=dict(),kwds=dict(),show=9):
    if 'min_cluster_size' not in kwds.keys():
        kwds['min_cluster_size'] = int(len(data)*0.01)
        kwds['min_samples'] = max(kwds['min_cluster_size']/5,2)
    
    labels = getClusters(data, kwds)
    shapes = prepClusters(data,labels)
    data['label'] = labels
    
    show = min(show,len(shapes))+1
    show -= 1
    
    indices = plotClusters(shapes,data,show=show)
    dispKey = {shapes.index[i]:shapes['keys'][i] for i in range(show)}
    textChunks = getTextSubsets(tweets,data,dispKey)
    return textChunks
    
    plt.show()
    print indices
    return shapes

runClusters(lymeData,biteLocs,kwds={'metric':'manhattan'},show=5)[0]

lat        lon       UTCTime  label  \
id                                                              
473628884873670600  39.449500 -75.716320  16224.246528      2   
473655818903375800  38.973966 -83.448263  16224.320845      2   
473658232230404000  39.045750 -76.641270  16224.327512      2   
473660482831011800  39.045750 -76.641270  16224.333715      2   
473665505531015100  37.455810 -77.617143  16224.347581      2   
473666583832043500  36.858640 -82.783240  16224.350556      2   
473674521963405300  39.045750 -76.641270  16224.372465      2   
473703255688495100  39.202280 -83.611500  16224.451748      2   
473716879761354700  36.850760 -76.285870  16224.489352      2   
473728516161146800  39.045750 -76.641270  16224.521458      2   
473823908576387000  39.045750 -76.641270  16224.784687      2   
473836726595911600  37.540720 -77.436040  16224.820058      2   
473843100537073600  36.974280 -76.367000  16224.837650      2   
473871526681649100  37.270970 -79.941420  16224.916088      2   
473879838512910300  39.107330 -76.571070  16224.939028      2   
473888524975210400  38.442160 -76.743970  16224.962998      2   
473896964971433900  39.290380 -76.612180  16224.986285      2   
473906643584180200  38.658170 -77.249700  16225.012998      2   
473912255583895500  37.431570 -78.656890  16225.028484      2   
473913501350912000  39.043750 -77.487440  16225.031921      2   
473913828397563900  37.524900 -76.824770  16225.032824      2   
473922451878195200  38.391008 -76.612762  16225.056620      2   
473953651396841400  37.442226 -82.115679  16225.142708      2   
473961828762918900  39.371001 -76.683328  16225.165278      2   
473966210988507100  39.045750 -76.641270  16225.177373      2   
473968733082226600  39.427300 -75.234070  16225.184329      2   
473973608704790500  39.045750 -76.641270  16225.197778      2   
473973631295311800  37.270970 -79.941420  16225.197847      2   
473986631418916800  37.540720 -77.436040  16225.233715      2   
473991179818635200  37.431570 -78.656890  16225.246273      2   
...                       ...        ...           ...    ...   
501788241230831600  38.324290 -76.516890  16301.951551      2   
501813994882351100  39.290380 -76.612180  16302.022616      2   
501947118333231100  38.851198 -76.632867  16302.389965      2   
502072786995531700  36.835420 -76.298270  16302.736748      2   
502125630474158000  37.496390 -78.244860  16302.882569      2   
502130389365039100  38.907190 -77.036870  16302.895694      2   
502187049353768900  36.850760 -76.285870  16303.052049      2   
502358991767605200  38.423140 -82.429270  16303.526516      2   
502410068068167600  39.290380 -76.612180  16303.667465      2   
502473748805459900  39.290380 -76.612180  16303.843183      2   
502604150874861500  37.655370 -82.783830  16304.203032      2   
502881131004723200  38.948935 -77.079354  16304.967350      2   
502882451325464500  38.895422 -77.009332  16304.970995      2   
502965818456821700  38.324290 -76.516890  16305.201042      2   
502991915764441000  39.290380 -76.612180  16305.273056      2   
503029213038133200  38.821851 -77.607639  16305.375972      2   
503423339126599600  38.990660 -77.026000  16306.463553      2   
503638135914639300  38.888600 -77.004700  16307.056285      2   
503696998877659100  38.907190 -77.036870  16307.218715      2   
504057577920266200  37.540720 -77.436040  16308.213727      2   
504065244713742300  38.846220 -77.306370  16308.234884      2   
504084622423375800  37.755500 -77.480840  16308.288356      2   
504100774222512100  38.232388 -77.538440  16308.332917      2   
504635901306474400  38.804830 -77.046920  16309.809595      2   
504636946816454600  38.804830 -77.046920  16309.812477      2   
505446468741001200  36.852920 -75.977900  16312.046331      2   
505550557646688200  37.379402 -77.437627  16312.333565      2   
506641066893012900  38.921439 -76.846564  16315.342789      2   
506916391027744700  38.936200 -77.553820  16316.102546      2